# textgenrnn

* https://github.com/minimaxir/textgenrnn
* https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb

In [1]:
import sys, os
import pandas as pd

In [7]:
data_folder = '/media/student/8d1913cf-1155-47a5-a7db-b9a51f445d8f/student/data'

In [8]:
os.path.exists(data_folder)

True

In [9]:
os.listdir(data_folder)

['yelp_dataset',
 'business.csv',
 'jigsaw-toxic-comment-classification-challenge.zip',
 'restaurants.csv',
 'restaurant_reviews.csv',
 'review.csv']

In [13]:
reviews = pd.read_csv(
    data_folder + '/restaurant_reviews.csv',
    nrows=10000,
    sep=',',
    index_col=0,
    usecols=['stars', 'text']
)

In [14]:
reviews.reset_index(inplace=True)

In [15]:
reviews.head()

,stars,text
0,5.0,Went in for a lunch. Steak sandwich was delici...
1,4.0,I'll be the first to admit that I was not exci...
2,3.0,Tracy dessert had a big name in Hong Kong and ...
3,1.0,This place has gone down hill. Clearly they h...
4,4.0,"Like walking back in time, every Saturday morn..."


In [9]:
reviews.head()

,Unnamed: 0,business_id,stars,text
0,3,ikCg8xy5JIg_NGPx-MSIDA,5.0,Went in for a lunch. Steak sandwich was delici...
1,5,eU_713ec6fTGNO4BegRaww,4.0,I'll be the first to admit that I was not exci...
2,6,3fw2X5bZYeW9xCz_zGhOHg,3.0,Tracy dessert had a big name in Hong Kong and ...
3,7,zvO-PJCpNk4fgAVUnExYAA,1.0,This place has gone down hill. Clearly they h...
4,10,8mIrX_LrOnAqWsB5JrOojQ,4.0,"Like walking back in time, every Saturday morn..."


In [101]:
reviews[reviews['stars'].isin([4, 5])]['text'].values

array(["Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amount of dressing, and distributed perfectly across each leaf. I know I'm going on about the salad ... But it was perfect.\r\n\r\nDrink prices were pretty good.\r\n\r\nThe Server, Dawn, was friendly and accommodating. Very happy with her.\r\n\r\nIn summation, a great pub experience. Would go again!",
       'I\'ll be the first to admit that I was not excited about going to La Tavolta. Being a food snob, when a group of friends suggested we go for dinner I looked online at the menu and to me there was nothing special and it seemed overpriced.  Im also not big on ordering pasta when I go out. Alas, I was outnumbered. Thank goodness! I ordered the sea bass special. It was to die for. Cooked perfectly, seasoned perfectly, perfect portion. I can not say enough good things about this dish. When the server asked how it was he seemed very proud of the dish and s

In [10]:
reviews.shape

(100000, 4)

In [11]:
from textgenrnn import textgenrnn
textgen = textgenrnn()

Using TensorFlow backend.


In [12]:
dir(textgen)

['META_TOKEN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'config',
 'default_config',
 'encode_text_vectors',
 'generate',
 'generate_samples',
 'generate_to_file',
 'indices_char',
 'load',
 'model',
 'num_classes',
 'reset',
 'save',
 'similarity',
 'tokenizer',
 'train_from_file',
 'train_from_largetext_file',
 'train_new_model',
 'train_on_texts',
 'vocab']

In [22]:
stars = [1, 2, 3, 4, 5]

In [23]:
texts = {
    star: reviews[reviews.stars == star]['text'].values[:20]
    for star in stars
}

In [27]:
models_pretrained = {
    star: textgenrnn(name='model_pretrained_{}'.format(star))
    for star in stars
}

In [28]:
models_new = {
    star: textgenrnn(name='model_new_{}'.format(star))
    for star in stars
}

In [33]:
def train_models(word_level, star2model, star2texts, num_epochs=1, gen_epochs=1, max_length=10, **kwargs):
    for star, model in star2model.items():
        model.train_on_texts(
            star2texts[star],
            word_level=word_level,
            num_epochs=num_epochs,
            gen_epochs=gen_epochs,
            max_length=max_length,
            **kwargs)

In [34]:
def train_models_word_level(star2model, star2texts, num_epochs=10, gen_epochs=5, max_length=40):
    train_models(True,
                 star2model,
                 star2texts,
                 num_epochs,
                 gen_epochs,
                 max_length,
                 **kwargs)

In [ ]:
def train_models_char_level(star2model, star2texts, num_epochs=80, gen_epochs=10, max_length=10, **kwargs):
    train_models(False,
                 star2model,
                 star2texts,
                 num_epochs,
                 gen_epochs,
                 max_length
                 **kwargs)

In [35]:
train_models_word_level(
    models_pretrained,
    texts,
    num_epochs=10,
    gen_epochs=5)

Training on 13,829 character sequences.
Epoch 1/10
108/108 [==============================] - 22s 201ms/step - loss: 1.1056
Epoch 2/10
108/108 [==============================] - 26s 244ms/step - loss: 0.9972
Epoch 3/10
108/108 [==============================] - 26s 239ms/step - loss: 0.8980
Epoch 4/10
108/108 [==============================] - 26s 238ms/step - loss: 0.8074
Epoch 5/10
108/108 [==============================] - 24s 224ms/step - loss: 0.7194
####################
Temperature: 0.2
####################
I would have also sandwich in a burrito and the food was a bean and said it.  The server was some of the sandwich. I would have also me that the food was almost done and the servers always have also service, so the service was their price and the food was pretty for a burrito. I would have also me 

I would have also several menu for a burrito. Well of it was and so many other prinks. I pull our orders, we had a burrito and she was and my friend and I wanted all of the menu.  

Just gross. Wenten this popuuter wead on my food.  A But Griten eaten I don't even was awesome slice of my fresh kano steaks. It has not so he good apologe. We had to fee the skewapes and for 3 mession from prices on the tatootao raw.

Every John Juct fork ( underwhelming)..vistent flavor. I sashilume which it was nothing to take our too. He wasn't someone else the respontive epard steaks. They though I alrechase 20 earleadles and restounes in dinner and watch, looked task when we said are has kids placie every high accommtdi whe

REALLY completely as why that the salabian waitress top (who will be a song. To completely no in filled unurl of sean which were good as effective he was eating at not the plater and when which shes had a great. The chemsieaces will not be not guy foe deskan hooks convension, we shouldn't have lebs

Epoch 6/10
140/140 [==============================] - 35s 247ms/step - loss: 0.8927
Epoch 7/10
140/140 [==============================] - 34s 246ms/step - loss: 0

I love this place and a few misses of mango service and a server, and I have to say it was the manager for a salt of much side drinks. I also exterent it was the food when we had a table and the food was a little salty bit the price was the food was the food was the price was good and I was surpris

####################
Temperature: 0.5
####################
I came one the menu to the first first time chinesantly and a server, and the menu - Not Down My favourite in the service was good. We had to chatt and a few friendly menu to sat it is one of the shicks and the menu disappointed bar and a second layout. Maybe we don't eats the little chicken in the

I adolant that was like here a small burger you even were to be more fresh anything in a little service.  The server is a side of the experience. It was our serva and the tasted sure the season for the soup and some confused was the neighboring on the side drink with experiencing the side. It was a

We went to be most been a restaurant a

Epoch 1/10
 6/41 [===>..........................] - ETA: 29s - loss: 2.0450

KeyboardInterrupt: 

In [ ]:
train_models_word_level(
    models_new,
    texts,
    num_epochs=10,
    gen_epochs=5)

In [46]:
t = models_pretrained[1].generate(2, return_as_list=True)

100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


In [47]:
t

["Also don't believe when they're suppose to go to the beast and they said it.  They were almost the bar that particular drinks are rude. One of the most of the same way (like when the the order) may be only with the service with someh so I can get better for a burrito because liking for the same tim",
 'so miserpend is working card for back and the food was too for a burrito?? Also, this place does not worth all.  All sometimes they will never be it was not a burrito. Once we credo and their food and they would not could be the food about being this point and many personast experience. it cheese t']

In [52]:
models_pretrained[1].save()

In [89]:
tt = textgenrnn(name='model_pretrained_1')

In [90]:


tt.reset()

In [91]:
texts = ['Never gonna give you up, never gonna let you down',
            'Never gonna run around and desert you',
            'Never gonna make you cry, never gonna say goodbye',
            'Never gonna tell a lie and hurt you']

In [99]:
tt.train_on_texts(texts, num_epochs=1, new_model=True)

Training new model w/ 2-layer, 128-cell LSTMs
Training on 174 character sequences.
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 3.2612
####################
Temperature: 0.2
####################
nne   nnnnenen nnnn nnn   nn nn  n rn n nn  r n ne  nn a n  n  nN  n    ne n ene  n   nt n n nn     n   nn n       eiu neennnnnno nn nug    e   nnnn     nN  a nne n            ne     e  n en n n e n  n  nn   n     n   n oe n  nn  n  ennnn   n    nn     nn n nn   n n   n     nnd   en   ae   ne n    

n nnnnnn ne nnnn nnne    n   nnnn nn n nnnn  nu   n nnn  nn  en n nn e   en nr  nnan n  e        n  n    n      nr        n ngn    n gnnn    nn nn  ne   ln    e  n n   n  n   e    nn   no   nn   nn      n     a n      n        n  n      r n nrn      nnn nn  rn e r n   n en uN vn n n  n  n n  n gn n

nnn n en nnn ennn n nnn nn   n   nnn na n n e n    nnn   e n  nn    n e e n    nee      nn  nn  nna       e   en    nagn n  ne n  e ann n   nr n       n n n enn n n   n n  n n n na 

In [97]:
dir(tt)

['META_TOKEN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'config',
 'default_config',
 'encode_text_vectors',
 'generate',
 'generate_samples',
 'generate_to_file',
 'indices_char',
 'load',
 'model',
 'num_classes',
 'reset',
 'save',
 'similarity',
 'tokenizer',
 'train_from_file',
 'train_from_largetext_file',
 'train_new_model',
 'train_on_texts',
 'vocab']

In [98]:
tt.config

{'rnn_layers': 2,
 'rnn_size': 128,
 'rnn_bidirectional': False,
 'max_length': 40,
 'max_words': 10000,
 'dim_embeddings': 100,
 'word_level': False,
 'single_text': False,
 'name': 'model_pretrained_1'}

In [139]:
textgen.generate(1, max_gen_length=100)

I was something peopt this pot that could he wanted a reason and a brind of the service was always 

